In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
# import important libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset

In [ ]:
# Load the model and tokenizer
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

OSError: ayaht/finetunedflan_t5_large does not appear to have a file named config.json. Checkout 'https://huggingface.co/ayaht/finetunedflan_t5_large/tree/main' for available files.

In [ ]:
dataset = load_dataset("allenai/sciq", split="train")
dataset.shape

README.md:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.99M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/339k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

(11679, 6)

In [ ]:
df_train = dataset.to_pandas()

In [ ]:
# change the empty cell to null
import numpy as np

df_train.replace(r'^\s*$', np.nan, regex=True, inplace=True)  # Replace empty strings and whitespaces with NaN


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11679 entries, 0 to 11678
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question        11679 non-null  object
 1   distractor3     11679 non-null  object
 2   distractor1     11679 non-null  object
 3   distractor2     11679 non-null  object
 4   correct_answer  11679 non-null  object
 5   support         10481 non-null  object
dtypes: object(6)
memory usage: 547.6+ KB


In [ ]:
df_trained_cleaned = df_train.dropna()


In [ ]:
df_trained_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10481 entries, 0 to 11678
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question        10481 non-null  object
 1   distractor3     10481 non-null  object
 2   distractor1     10481 non-null  object
 3   distractor2     10481 non-null  object
 4   correct_answer  10481 non-null  object
 5   support         10481 non-null  object
dtypes: object(6)
memory usage: 573.2+ KB


In [ ]:
df_trained_renamed = df_trained_cleaned.rename(columns={'support': 'context', 'correct_answer': 'answer'})


In [ ]:
df_trained_renamed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10481 entries, 0 to 11678
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   question     10481 non-null  object
 1   distractor3  10481 non-null  object
 2   distractor1  10481 non-null  object
 3   distractor2  10481 non-null  object
 4   answer       10481 non-null  object
 5   context      10481 non-null  object
dtypes: object(6)
memory usage: 573.2+ KB


In [ ]:
df_trained_dropped = df_trained_renamed.drop(columns=['distractor1', 'distractor2', 'distractor3'])


In [ ]:
df_trained_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10481 entries, 0 to 11678
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  10481 non-null  object
 1   answer    10481 non-null  object
 2   context   10481 non-null  object
dtypes: object(3)
memory usage: 327.5+ KB


In [ ]:
from datasets import Dataset

dataset_train = Dataset.from_pandas(df_trained_dropped)

In [ ]:
dataset_validation = load_dataset("allenai/sciq", split="validation")
dataset_validation.shape

(1000, 6)

In [ ]:
df_val= dataset_validation.to_pandas()

In [ ]:
# change the empty cell to null
import numpy as np

df_val.replace(r'^\s*$', np.nan, regex=True, inplace=True)  # Replace empty strings and whitespaces with NaN


In [ ]:
df_val_cleaned = df_val.dropna()


In [ ]:
df_val_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 887 entries, 2 to 999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question        887 non-null    object
 1   distractor3     887 non-null    object
 2   distractor1     887 non-null    object
 3   distractor2     887 non-null    object
 4   correct_answer  887 non-null    object
 5   support         887 non-null    object
dtypes: object(6)
memory usage: 48.5+ KB


In [ ]:
df_val_renamed = df_val_cleaned.rename(columns={'support': 'context', 'correct_answer': 'answer'})


In [ ]:
df_val_dropped = df_val_renamed.drop(columns=['distractor1', 'distractor2', 'distractor3'])

In [ ]:
df_val_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 887 entries, 2 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  887 non-null    object
 1   answer    887 non-null    object
 2   context   887 non-null    object
dtypes: object(3)
memory usage: 27.7+ KB


In [ ]:
dataset_validation = Dataset.from_pandas(df_val_dropped)

In [ ]:
#datatrain --> dataset_train
#datavalidation --> dataset_validation

In [ ]:
# Preprocess function
def preprocess_function(examples):
    inputs = [f"Generate a question for this context: {context}" for context in examples["context"]]
    targets = [f"Question: {question}\nAnswer: {answer}" for question, answer in zip(examples["question"], examples["answer"])]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
# Preprocess the datasets
tokenized_train_dataset = dataset_train.map(preprocess_function, batched=True, remove_columns=dataset_train.column_names)
tokenized_val_dataset = dataset_validation.map(preprocess_function, batched=True, remove_columns=dataset_validation.column_names)


Map:   0%|          | 0/10481 [00:00<?, ? examples/s]

Map:   0%|          | 0/887 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=2, # Reduced batch size to 2
    per_device_eval_batch_size=2, # Reduced batch size to 2
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)


In [ ]:
# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
# Start training
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.000000,nan
2,0.000000,nan


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=10482, training_loss=0.0, metrics={'train_runtime': 6474.1218, 'train_samples_per_second': 3.238, 'train_steps_per_second': 1.619, 'total_flos': 4.831254578567578e+16, 'train_loss': 0.0, 'epoch': 2.0})

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./finetuned_flan_t5_large")
tokenizer.save_pretrained("./finetuned_flan_t5_large")

('./finetuned_flan_t5_large/tokenizer_config.json',
 './finetuned_flan_t5_large/special_tokens_map.json',
 './finetuned_flan_t5_large/spiece.model',
 './finetuned_flan_t5_large/added_tokens.json',
 './finetuned_flan_t5_large/tokenizer.json')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Determine the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the fine-tuned model and tokenizer
model_path = "./finetuned_flan_t5_large"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Move the model to the appropriate device
model = model.to(device)

def generate_question(context):
    input_text = f"Generate a question for this context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output = model.generate(input_ids, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def generate_answer(context, question):
    input_text = f"Answer this question based on the given context. Question: {question} Context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output = model.generate(input_ids, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def generate_qa(context):
    question = generate_question(context)
    answer = generate_answer(context, question)
    return question, answer

# Test the model
test_context = "Vertebrata are characterized by the presence of a backbone, such as the one that runs through the middle of this fish. All vertebrates are in the Craniata clade and have a cranium. (credit: Ernest V. More; taken at Smithsonian Museum of Natural History, Washington, D."

# Generate both question and answer
generated_question, generated_answer = generate_qa(test_context)
print("Generated Question:", generated_question)
print("Generated Answer:", generated_answer)



Using device: cuda
Generated Question: What is the name of the clade that all vertebrates are in?
Generated Answer: Craniata


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from huggingface_hub import HfApi

# Define repository details
repo_name = "finetuned_flan_t5"
model_repo_url = HfApi().create_repo(repo_id=repo_name, token=True)


In [ ]:
output_dir = "./finetuned_flan_t5_large"


In [ ]:
!rm -rf /content/finetuned_flan_t5_large/.git  # Removes the existing .git folder


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id="ayaht/flanfinetuned", exist_ok=True)
api.upload_folder(
    folder_path="/content/finetuned_flan_t5_large",
    repo_id="ayaht/flanfinetuned",
    repo_type="model"
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ayaht/flanfinetuned/commit/11337d80dfb127d46ecac21f12b661623a61989f', commit_message='Upload folder using huggingface_hub', commit_description='', oid='11337d80dfb127d46ecac21f12b661623a61989f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Load the model and tokenizer
model_name = "ayaht/flanfinetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]